In [ ]:
import os
import librosa
import librosa.display
import warnings
warnings.filterwarnings('ignore')
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kurtosis
from scipy.stats import skew
import csv

In [ ]:
# 파일 경로
# audio_path = "/content/drive/Othercomputers/내 노트북/Project/Dataset/songs/download/classic/converted"
audio_path = "/content/drive/Othercomputers/내 노트북/Project/Dataset/songs/download/converted/2022-03-21_"

In [ ]:
def song_names(audio_path):                      # audio_path : 음원 파일들의 폴더 경로
    files = list(os.listdir(f'{audio_path}'))

    songs = []
    paths = []

    for file in files:
        song = f'{file}'
        path = f'{audio_path}/{file}'
        songs.append(song)
        paths.append(path)

    return songs, paths

In [ ]:
def soundwave(paths):
    audio_files = []
    song_name = []

    for path in paths:
        try:
            p = path.split("/")[-1]
            y, sr = librosa.load(path, duration = 30)
            y, _ = librosa.effects.trim(y)
            song_name.append(p)
            audio_files.append(y)

        except ValueError as val:
            print(path, val)

    return audio_files, song_name, sr # sr 디폴트 : 22050 / 변경 가능한 sr 수치 : 44100

In [ ]:
def get_features(y, sr):
    # Features to concatenate in the final dictionary
    features = {'centroid': None, 'roloff': None, 'flux': None, 'rmse': None,
                'zcr': None, 'contrast': None, 'bandwidth': None, 'flatness': None, 'chroma_stft': None}
    
    # Count silence
    if 0 < len(y):
        y_sound, _ = librosa.effects.trim(y)
    features['sample_silence'] = len(y) - len(y_sound)

    # Using librosa to calculate the features
    features['chroma_stft']=librosa.feature.chroma_stft(y=y, sr=sr).ravel()
    features['centroid'] = librosa.feature.spectral_centroid(y, sr=sr).ravel()
    features['roloff'] = librosa.feature.spectral_rolloff(y, sr=sr,).ravel()
    features['zcr'] = librosa.feature.zero_crossing_rate(y).ravel()
    features['rmse'] = librosa.feature.rms(y).ravel()
    features['flux'] = librosa.onset.onset_strength(y=y, sr=sr).ravel()
    features['contrast'] = librosa.feature.spectral_contrast(y, sr=sr).ravel()
    features['bandwidth'] = librosa.feature.spectral_bandwidth(y, sr=sr).ravel()
    features['flatness'] = librosa.feature.spectral_flatness(y).ravel()
    
    # harmony, perceptral treatment (+@ 부분)
    harm, perc = librosa.effects.hpss(y)
    features['harm'] = harm.ravel()
    features['perc'] = perc.ravel()

    # MFCC treatment
    mfcc = librosa.feature.mfcc(y, sr=sr)   # n_mfcc=20 (default) 
    for idx, v_mfcc in enumerate(mfcc):
        features['mfcc_{}'.format(idx)] = v_mfcc.ravel()
        
    # Get statistics from the vectors
    def get_moments(descriptors):
        result = {}
        for k, v in descriptors.items():
            result['{}_max'.format(k)] = np.max(v)
            result['{}_min'.format(k)] = np.min(v)
            result['{}_mean'.format(k)] = np.mean(v)
            result['{}_std'.format(k)] = np.std(v)
            result['{}_kurtosis'.format(k)] = kurtosis(v)
            result['{}_skew'.format(k)] = skew(v)
        return result
    
    dict_agg_features = get_moments(features)
#    onset_env = librosa.onset.onset_strength(y=audio_file, sr=sr)
#    dict_agg_features['tempo'] = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)
    dict_agg_features['tempo'] = librosa.beat.tempo(y, sr=sr)[0]
    
    return dict_agg_features

In [ ]:
def final_extraction(dataset_dir=audio_path):
#    genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
    last_features=[]

    songs, paths = song_names(audio_path)
    audio_files, song_name, sr = soundwave(paths)

    for audio in audio_files:
        features = get_features(audio, sr)
        last_features.append(features)

    return last_features, songs

In [ ]:
%%time

last_features, songs = final_extraction()

CPU times: user 1h 18min 14s, sys: 9min 17s, total: 1h 27min 31s
Wall time: 1h 19min 22s


In [ ]:
def df_all_features(last_features, songs):
    df_features = pd.DataFrame(last_features)
    df_songs = pd.DataFrame(songs, columns=['song'])

    return pd.concat([df_songs, df_features], axis=1)

In [ ]:
df_all_features = df_all_features(last_features, songs)
df_all_features.head()

,song,centroid_max,centroid_min,centroid_mean,centroid_std,centroid_kurtosis,centroid_skew,roloff_max,roloff_min,roloff_mean,...,mfcc_18_std,mfcc_18_kurtosis,mfcc_18_skew,mfcc_19_max,mfcc_19_min,mfcc_19_mean,mfcc_19_std,mfcc_19_kurtosis,mfcc_19_skew,tempo
0,Timeless-SG 워너비_30.wav,6808.391284,779.971841,2881.654588,917.021626,0.710205,0.273543,9517.675781,1108.959961,6160.937746,...,10.729717,0.281442,-0.276641,23.558681,-29.480442,-1.841570,8.568623,-0.128778,0.222978,107.666016
1,포장마차-황인욱_30.wav,6528.036158,936.409533,2297.386382,913.281119,5.989205,2.199445,10109.838867,1582.690430,4773.054471,...,10.628619,0.645889,-0.242336,34.233147,-31.697285,-3.113328,10.869784,0.237007,0.402205,129.199219
2,너의 번호를 누르고 (Prod. 영화처럼)-#안녀...,6407.063051,687.534543,1853.074627,721.904316,11.407568,2.922212,9603.808594,796.728516,3839.901781,...,10.979051,-0.505732,0.011383,33.469559,-19.840303,4.284270,9.211324,0.051515,0.435930,135.999178
3,다시 만날까 봐-V.O.S_30.wav,6318.793047,444.777547,1870.412089,835.083193,2.463315,1.083184,9765.307617,570.629883,4062.625434,...,9.571832,0.153035,0.213881,37.991161,-30.633295,3.197549,11.832664,0.275204,0.237370,129.199219
4,혼자 왔어요-디셈버_30.wav,5944.306961,604.365330,1700.478342,773.921127,2.298078,1.078844,9851.440430,839.794922,3963.126036,...,9.094260,1.156862,0.010668,30.870239,-33.500244,-0.190033,10.220849,0.346638,0.194412,135.999178


In [ ]:
# export the data to a csv file
df_all_features.to_csv('/content/drive/Othercomputers/내 노트북/Project/Models/Music_Genre_Classification_Recommendation/Feature_Result_csv/feature_result.csv')

In [ ]:
# import the data
df_all_features = pd.read_csv('/content/drive/Othercomputers/내 노트북/Project/Models/Music_Genre_Classification_Recommendation/Feature_Result_csv/feature_result.csv', index_col = 0)

In [ ]:
df_all_features

,song,centroid_max,centroid_min,centroid_mean,centroid_std,centroid_kurtosis,centroid_skew,roloff_max,roloff_min,roloff_mean,...,mfcc_18_std,mfcc_18_kurtosis,mfcc_18_skew,mfcc_19_max,mfcc_19_min,mfcc_19_mean,mfcc_19_std,mfcc_19_kurtosis,mfcc_19_skew,tempo
0,Timeless-SG 워너비_30.wav,6808.391284,779.971841,2881.654588,917.021626,0.710205,0.273543,9517.675781,1108.959961,6160.937746,...,10.729717,0.281442,-0.276641,23.558681,-29.480442,-1.841570,8.568623,-0.128778,0.222978,107.666016
1,포장마차-황인욱_30.wav,6528.036158,936.409533,2297.386382,913.281119,5.989205,2.199445,10109.838867,1582.690430,4773.054471,...,10.628619,0.645889,-0.242336,34.233147,-31.697285,-3.113329,10.869784,0.237007,0.402205,129.199219
2,너의 번호를 누르고 (Prod. 영화처럼)-#안녀...,6407.063051,687.534543,1853.074627,721.904316,11.407568,2.922212,9603.808594,796.728516,3839.901781,...,10.979051,-0.505732,0.011383,33.469560,-19.840303,4.284270,9.211324,0.051515,0.435930,135.999178
3,다시 만날까 봐-V.O.S_30.wav,6318.793047,444.777547,1870.412089,835.083193,2.463315,1.083184,9765.307617,570.629883,4062.625434,...,9.571832,0.153035,0.213881,37.991160,-30.633295,3.197549,11.832664,0.275204,0.237370,129.199219
4,혼자 왔어요-디셈버_30.wav,5944.306961,604.365330,1700.478342,773.921127,2.298078,1.078844,9851.440430,839.794922,3963.126036,...,9.094260,1.156862,0.010668,30.870240,-33.500244,-0.190033,10.220849,0.346638,0.194412,135.999178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1112,사랑이란 멜로는 없어-전상근_30.wav,5871.307449,652.686946,1887.116110,712.621507,8.066007,2.436668,9614.575195,872.094727,3723.044154,...,13.192509,-0.321352,0.215080,42.287186,-27.373474,3.863741,11.266271,-0.175128,0.234996,123.046875
1113,가슴 아파도-환희_30.wav,5059.006502,614.032706,1580.992107,662.418900,3.350908,1.530171,8021.118164,818.261719,3395.287802,...,9.743752,1.170244,-0.010512,21.113426,-31.827812,-2.086408,8.925055,0.275026,-0.396901,129.199219
1114,사랑은...향기를 남기고-테이_30.wav,6036.061736,613.475149,2320.967429,1029.832063,1.733287,1.064968,9130.078125,904.394531,5091.960876,...,9.238767,1.228230,0.776293,35.749740,-20.205238,2.707291,9.719677,-0.142164,0.365823,129.199219
1115,사랑이 다른 사랑으로 잊혀지네-하림_30.wav,5820.610782,422.024925,1626.044880,647.161001,6.830122,1.807256,8602.514648,495.263672,3756.785617,...,8.189332,0.012250,0.235152,38.079872,-23.583622,0.926317,9.150774,0.479233,0.305111,89.102909


### 시도해봐야 하는 것
- 에러 로그 찍기
- df에 곡명 컬럼 추가하기

In [ ]:
# list(df_features.columns)